In [2]:
!pip install pandas
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re

In [192]:
# Set up Selenium WebDriver (you need to have appropriate drivers installed)
driver = webdriver.Chrome()  # You can use other browsers and corresponding drivers too

# Load the page
driver.get("https://batdongsan.com.vn/ban-nha-dat-ha-noi")

# Wait for the page to load completely (you might need to adjust the time)
driver.implicitly_wait(10)

# Get the page source after JavaScript has rendered the content
page_source = driver.page_source

# Now you can use BeautifulSoup to parse the page source
soup = BeautifulSoup(page_source, "html.parser")

# Find the link
anchor_tags = soup.findAll('a', 'js__product-link-for-product-id')
anchor_tag = anchor_tags[0]

# Extract the href attribute
if anchor_tag:
    href = anchor_tag.get('href')
    print(href)
else:
    print("No anchor tag found.")

# Close the browser
driver.quit()

/ban-nha-mat-pho-duong-bui-thi-xuan-phuong-bui-thi-xuan/ban-mp-vip-o-to-tranh-via-he-kinh-doanh-1-so-1-chu-no-hau-vuong-phong-thuy-pr39846461


In [176]:
# Initialize an empty list to store links
LIST_LINK_PRODUCT = []

try:
    for i in range(860, 1000):  # Adjust the range as needed
        # Set up Selenium WebDriver
        driver = webdriver.Chrome() 

        url = "https://batdongsan.com.vn/ban-nha-dat-ha-noi/p" + str(i + 1)
        driver.get(url)

        # Get the page source
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, "html.parser")
        list_link = soup.findAll('a', 'js__product-link-for-product-id')

        if not list_link:
            print(f'####Error: No links found on page {i + 1}')
            driver.quit()  # Quit the WebDriver if no links found
            continue

        # Extract links and append them to the list
        for item in list_link:
            link_href = item.get('href')
            LIST_LINK_PRODUCT.append({'link': 'https://batdongsan.com.vn/' + link_href})

        print(f'####Done page {i + 1}')

        # Close the browser
        driver.quit()

except Exception as e:
    print(f'####Error getting links from page {i + 1}')
    print(e)

####Done page 861
####Done page 862
####Done page 863
####Done page 864
####Done page 865
####Done page 866
####Done page 867
####Done page 868
####Done page 869
####Done page 870
####Done page 871
####Done page 872
####Done page 873
####Done page 874
####Done page 875
####Done page 876
####Done page 877
####Done page 878
####Done page 879
####Done page 880
####Done page 881
####Done page 882
####Done page 883
####Done page 884
####Done page 885
####Done page 886
####Done page 887
####Done page 888
####Done page 889
####Done page 890
####Done page 891
####Done page 892
####Done page 893
####Done page 894
####Done page 895
####Done page 896
####Done page 897
####Done page 898
####Done page 899
####Done page 900
####Done page 901
####Done page 902
####Done page 903
####Done page 904
####Done page 905
####Done page 906
####Done page 907
####Done page 908
####Done page 909
####Done page 910
####Done page 911
####Done page 912
####Done page 913
####Done page 914
####Done page 915
####Done p

In [13]:
# print(LIST_LINK_PRODUCT)
# df = pd.DataFrame(LIST_LINK_PRODUCT)
# df.to_csv('data/link_data.csv')

df = pd.DataFrame(LIST_LINK_PRODUCT)
existing_df = pd.read_csv('data/link_data.csv')
last_index = existing_df.index.max()
df.index = range(last_index + 1, last_index + 1 + len(df))
df.to_csv('data/link_data.csv', mode='a', header=False, index=True)

In [227]:
# Set up Selenium WebDriver
driver = webdriver.Chrome() 

url = "https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-chi-thanh-phuong-lang-thuong/ban-ngo-76-45m-4-tang-5-45-ty-dong-da-pr39837906"
driver.get(url)

# Initialize a dictionary to store property details with default values
property_details = {
    'ward': '',
    'district': '',
    'price_per_sqm': '',
    'posted_date': '',
    'area': '',
    'price': '',
    'legal_document': '',
    'interior': '',
    'num_bedrooms': '',
    'num_bathrooms': '',
    'num_floors': '',
    'house_orientation': '',
    'balcony_orientation': '',
    'entrance': '',
    'frontage': ''
}

# Set up Selenium WebDriver
with webdriver.Chrome() as driver:
    driver.get(url)
    
    # Get the page source
    page_source = driver.page_source

soup = BeautifulSoup(page_source, "html.parser")
with open('data/test.html', 'w') as file:
    file.write(soup.prettify())

# Split the address into parts
address_parts = soup.find('span', class_='re__pr-short-description').text.split(', ')


# Extract ward and district
ward = address_parts[-3]
property_details['ward'] = ward
district = address_parts[-2]
property_details['district'] = district

# Find the span containing the price per square meter
price_per_sqm_span = soup.find('span', class_='ext')

# Extract the text from the span
price_per_sqm = price_per_sqm_span.text.strip()

property_details['price_per_sqm'] = price_per_sqm

specs_content = soup.findAll('div', 're__pr-specs-content-item')

# Find the span containing the posted date
posted_date_span = soup.find('span', string='Ngày đăng')

# Extract the value of the posted date
posted_date = posted_date_span.find_next_sibling('span').text.strip()
property_details['posted_date'] = posted_date


# Mapping of titles to corresponding keys in property_details
title_map = {
    'Diện tích': 'area',
    'Mức giá': 'price',
    'Giấy tờ phjáp lý': 'legal_document',
    'Nội thất': 'interior',
    'Số phòng ngủ': 'num_bedrooms',
    'Số phòng tắm, vệ sinh': 'num_bathrooms',
    'Số tầng': 'num_floors',
    'Hướng nhà': 'house_orientation',
    'Hướng ban công': 'balcony_orientation',
    'Đường vào': 'entrance',
    'Mặt tiền': 'frontage'
}

# Extract and map property details
for item in specs_content:
    title = item.find(class_='re__pr-specs-content-item-title').text.strip()
    value = item.find(class_='re__pr-specs-content-item-value').text.strip()
    if title in title_map:
        property_details[title_map[title]] = value

print(property_details)

{'ward': 'Phường Láng Thượng', 'district': 'Đống Đa', 'price_per_sqm': '~146,25 triệu/m²', 'posted_date': '13/05/2024', 'area': '40 m²', 'price': '5,85 tỷ', 'legal_document': '', 'interior': '', 'num_bedrooms': '4 phòng', 'num_bathrooms': '', 'num_floors': '4 tầng', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}


In [14]:
df_link_data = pd.read_csv('data/link_data.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20000 non-null  int64 
 1   link        20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [15]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()
print(len(LIST_LINK_PRODUCT))

20000


In [16]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'type_estate': '',
        'district': '',
        'price_per_sqm': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    # Set up Selenium WebDriver
    with webdriver.Chrome() as driver:
        driver.get(url)
        # Get the page source
        page_source = driver.page_source

    soup = BeautifulSoup(page_source, "html.parser")

    # Find the breadcrumb element
    breadcrumb = soup.find('div', class_='re__breadcrumb')

    level_4_link = breadcrumb.find('a', {'level': '4'})
    level_4_title = level_4_link.text

    # Use regular expressions to extract the type_estate
    match = re.search(r"(.+?) tại", level_4_title)
    type_estate = match.group(1) if match else ""
    property_details['type_estate'] = type_estate

    level_3_link = breadcrumb.find('a', {'level': '3'})
    property_details['district'] = level_3_link.text

    # Extract price per square meter
    price_per_sqm_span = soup.find('span', class_='ext')
    if price_per_sqm_span:
        property_details['price_per_sqm'] = price_per_sqm_span.text.strip()

    # Extract posted date
    posted_date_span = soup.find('span', string='Ngày đăng')
    property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()

    # Extract property specifications
    specs_content = soup.find_all('div', class_='re__pr-specs-content-item')

    # Mapping of titles to corresponding keys in property_details
    title_map = {
        'Diện tích': 'area',
        'Mức giá': 'price',
        'Pháp lý': 'legal_document',
        'Nội thất': 'interior',
        'Số phòng ngủ': 'num_bedrooms',
        'Số toilet': 'num_bathrooms',
        'Số tầng': 'num_floors',
        'Hướng nhà': 'house_orientation',
        'Hướng ban công': 'balcony_orientation',
        'Đường vào': 'entrance',
        'Mặt tiền': 'frontage'
    }

    # Extract and map property details
    for item in specs_content:
        title = item.find(class_='re__pr-specs-content-item-title').text.strip()
        if title in title_map:
            property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
    
    return property_details

# Iterate through LIST_LINK_PRODUCT and append results to LIST_PRODUCT
for index, link in enumerate(LIST_LINK_PRODUCT):
    try:
        print(f"Scanning URL {index + 1}/{len(LIST_LINK_PRODUCT)}: {link}")
        details = extract_property_details(link)
        LIST_PRODUCT.append(details)
        print(f"Extracted details: {details}")
    except Exception as e:
        print(f"Failed to extract data for {link}: {e}")
        break  # Stop further processing to check the issue

# Now LIST_PRODUCT contains the extracted details for all successfully processed URLs
print(LIST_PRODUCT)

Scanning URL 1/20000: https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-duong-le-trong-tan-phuong-duong-noi-prj-khu-do-thi-geleximco-le-trong-tan/chinh-chu-ban-can-dep-c49-o20-kdt-geximco-80m-4-g-hoan-thien-vip-pr39840842
Extracted details: {'type_estate': 'Nhà biệt thự, liền kề', 'district': 'Hà Đông', 'price_per_sqm': '~173,12 triệu/m²', 'posted_date': '13/05/2024', 'area': '80 m²', 'price': '13,85 tỷ', 'legal_document': 'Sổ đỏ/ Sổ hồng', 'interior': 'Đầy đủ', 'num_bedrooms': '6 phòng', 'num_bathrooms': '4 phòng', 'num_floors': '4 tầng', 'house_orientation': '', 'balcony_orientation': '', 'entrance': '', 'frontage': ''}
Scanning URL 2/20000: https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-duong-tran-phu-phuong-mo-lao/cc-chuyen-nhuong-625m-san-kinh-doanh-mat-ha-dong-so-do-lau-dai-23-ty-pr39824404
Extracted details: {'type_estate': 'Shophouse, nhà phố thương mại', 'district': 'Hà Đông', 'price_per_sqm': '~36,8 triệu/m²', 'posted_date': '11/05/2024', 'area': '625 m²', 'price

In [ ]:
print(len(LIST_PRODUCT))

4000


In [ ]:
df_link_product = pd.DataFrame(LIST_PRODUCT)
df_link_product.to_csv('data/raw_data.csv')

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("data/raw_data.csv")

# Drop the 'link' and 'describe' columns
df.drop(['link', 'describe'], axis=1, inplace=True)

# Save the modified DataFrame back to a CSV file
df.to_csv("data/raw_data.csv", index=False)

In [ ]:
df = pd.read_csv('data/raw_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    4000 non-null   int64  
 1   estate_type   3210 non-null   object 
 2   province      3210 non-null   object 
 3   district      3206 non-null   object 
 4   ward          3206 non-null   object 
 5   price         3824 non-null   object 
 6   square        3770 non-null   object 
 7   post_date     4000 non-null   object 
 8   numb_bedroom  3076 non-null   float64
 9   numb_toilet   2864 non-null   float64
 10  numb_floor    3155 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 343.9+ KB
